In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
from rnn.AWD_LSTM import AWDLSTM
import urllib.request
import os
import unidecode
import string
import random
import re
import time
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import CosineAnnealingLR
from rnn.helpers import time_since
from rnn.generate import generate
%matplotlib inline
import pdb
%load_ext autoreload
%autoreload 2

In [2]:
import torch
print(torch.__version__)

1.10.2+cu111


In [3]:
# vocab = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .,:;'\"-+!?()/\\[]{}<>"
# vocab_mapping = {char: i for i, char in enumerate(vocab)}

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
# Download text file if it does not exist
url = "https://www.gutenberg.org/files/11/11-0.txt"
filename = "language_data/input.txt"
if not os.path.isfile(filename):
    urllib.request.urlretrieve(url, filename)

# Load and preprocess text
file_path = 'language_data/input.txt'
file = unidecode.unidecode(open(file_path).read())
all_characters = string.printable
file_chars = [c for c in file if c in all_characters]
char_to_idx = {char: i for i, char in enumerate(all_characters)}
idx_to_char = {i: ch for i, ch in enumerate(all_characters)}
vocab_size = len(all_characters)

# Split data into train and test sets
split = int(0.9 * len(file_chars))
train_text = file_chars[:split]
test_text = file_chars[split:]

print('train len: ', len(train_text))
print('test len: ', len(test_text))
print('All characters: ', all_characters)

chunk_len = 200

train len:  11842726
test len:  1315859
All characters:  0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	



In [25]:
import urllib.request

# Download the WikiText-2 dataset
url = "https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip"
urllib.request.urlretrieve(url, "wikitext-2-v1.zip")

# Extract the dataset
import zipfile
with zipfile.ZipFile("wikitext-2-v1.zip", "r") as zip_ref:
    zip_ref.extractall()

# Read in the text from the dataset
with open("wikitext-2/wiki.test.tokens", "r") as f:
    text = f.read()

# Get the first 1000 characters
text_1000 = text[:1000]

print(text_1000)

 
 = Robert <unk> = 
 
 Robert <unk> is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John <unk> in 2002 . In 2004 <unk> landed a role as " Craig " in the episode " Teddy 's Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the <unk> <unk> Factory in London . He was directed by John <unk> and starred alongside Ben <unk> , Shane <unk> , Harry Kent , Fraser <unk> , Sophie Stanton and Dominic Hall . 
 In 2006 , <unk> starred alongside <unk> in the play <unk> written by Mark <unk> . He appeared on a 2006 episode of the televisio

In [6]:
chunk_len = 200

def random_chunk(text):
    start_index = random.randint(0, len(text) - chunk_len)
    end_index = start_index + chunk_len + 1
    return text[start_index:end_index]

print(random_chunk(train_text))

['o', 'f', 'f', ' ', '.', ' ', 'I', 't', ' ', 'c', 'e', 'a', 's', 'e', 's', ' ', 'a', 'l', 'm', 'o', 's', 't', ' ', 'e', 'n', 't', 'i', 'r', 'e', 'l', 'y', ' ', 'n', 'e', 'a', 'r', ' ', 't', 'h', 'e', ' ', 'n', 'o', 'r', 't', 'h', 'e', 'r', 'n', ' ', 'c', 'i', 't', 'y', ' ', 'l', 'i', 'n', 'e', ' ', 'a', 't', ' ', 'Y', 'o', 'r', 'k', ' ', 'S', 't', 'r', 'e', 'e', 't', ' ', ',', ' ', 'w', 'h', 'e', 'r', 'e', ' ', 't', 'h', 'e', ' ', 'h', 'o', 'm', 'e', 's', ' ', 'a', 'l', 'o', 'n', 'g', ' ', 't', 'h', 'e', ' ', 'h', 'i', 'g', 'h', 'w', 'a', 'y', ' ', 'b', 'e', 'c', 'o', 'm', 'e', ' ', 'm', 'o', 'r', 'e', ' ', 's', 'p', 'o', 'r', 'a', 'd', 'i', 'c', ' ', 'a', 'n', 'd', ' ', 's', 'p', 'a', 'c', 'e', 'd', ' ', 'a', 'p', 'a', 'r', 't', ' ', '.', ' ', '\n', ' ', '\n', ' ', '=', ' ', '=', ' ', '=', ' ', '=', ' ', 'N', 'o', 'r', 't', 'h', ' ', 'o', 'f', ' ', 'A', 'u', 'b', 'u', 'r', 'n', ' ', '=', ' ', '=', ' ', '=', ' ', '=', ' ', '\n', ' ', '\n', ' ', 'N', 'o', 'w', ' ', 'i', 'n', ' ', 't', 

In [7]:
def char_tensor(string, batch_size=1):
    if isinstance(string, str):
        string = [string]
    tensor = torch.zeros(len(string), batch_size).long()
    for b in range(batch_size):
        for c in range(len(string)):
            char = string[c][b]
            if char in char_to_idx:
                tensor[c][b] = char_to_idx[char]
    return tensor


def load_random_batch(text, seq_length, batch_size):
    inputs = torch.zeros(seq_length, batch_size).long()
    targets = torch.zeros(seq_length, batch_size).long()

    for i in range(batch_size):
        start_index = random.randint(0, len(text) - seq_length)
        end_index = start_index + seq_length + 1
        chunk = text[start_index:end_index]
        inputs[:, i] = char_tensor(chunk[:-1], batch_size=1)[:, 0]
        targets[:, i] = char_tensor(chunk[1:], batch_size=1)[:, 0]

    return inputs, targets

def text_to_tensor(text, vocab_mapping):
    indices = [vocab_mapping[char] for char in text]
    return torch.tensor(indices, dtype=torch.long)

In [20]:
def generate(model, prime_str='A', predict_len=100, temperature=0.8, device='cpu'):
    hidden = (torch.zeros(model.num_layers, 2, model.hidden_size, device=device),
              torch.zeros(model.num_layers, 2, model.hidden_size, device=device))
    prime_input = torch.tensor([char_to_idx[c] for c in prime_str], dtype=torch.long, device=device)
    generated_text = prime_str

    with torch.no_grad():
        for i in range(predict_len):
            output, hidden = model(prime_input.view(1, -1), hidden)
            output_dist = output.squeeze().div(temperature).exp()
            top_i = torch.multinomial(output_dist, 1)[0]
            generated_char = idx_to_char[top_i]
            generated_text += generated_char
            prime_input.fill_(top_i)

    return generated_text



In [9]:
def eval_test(rnn, inp, target):
    with torch.no_grad():
        hidden = rnn.init_hidden(inp.shape[1], device=device)
        loss = 0
        for c in range(chunk_len):
            output, hidden = rnn(inp[:,c], hidden)
            loss += criterion(output.view(inp.shape[1], -1), target[:,c])
    
    return loss.data.item() / chunk_len

In [10]:
# Hyperparameters
input_size = vocab_size
embedding_size = 300
hidden_size = 1024
num_layers = 10
dropout = 0.05
weight_dropout = 0.05
batch_size = 1024
seq_length = 1024
learning_rate = 0.001
num_epochs = 5000
bptt=2000

In [14]:
# Define the model, loss function, optimizer, and learning rate scheduler
input_size = len(all_characters)
awd_lstm = AWDLSTM(input_size, hidden_size, num_layers, embedding_size=embedding_size, dropout=dropout, weight_dropout=weight_dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(awd_lstm.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.99, patience=100, verbose=True)

# Split data into train and validation sets
split = int(0.9 * len(train_text))
train_text = file_chars[:split]
valid_text = file_chars[split:]

print('train len: ', len(train_text))
print('valid len: ', len(valid_text))

# Define data loaders
train_loader = DataLoader(train_text, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_text, batch_size=batch_size, shuffle=True)

# Training loop
awd_lstm.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    hidden = None
    num_batches = 0
    for batch_idx in range(0, (len(file) - seq_length) // (batch_size * seq_length), batch_size):
        input_batch, target_batch = load_random_batch(file[batch_idx*seq_length:(batch_idx+1)*seq_length], seq_length, batch_size)
        num_batches += 1

        for chunk_start in range(0, seq_length, bptt):
            chunk_input = char_tensor(input_batch[:, chunk_start:chunk_start + bptt]).to(device)
            chunk_target = char_tensor(target_batch[:, chunk_start:chunk_start + bptt]).to(device)

            # Forward pass
            output, hidden = awd_lstm(chunk_input, hidden)
            hidden = (hidden[0].detach(), hidden[1].detach())  # Detach hidden state to prevent backpropagation beyond the chunk
            output = output.view(-1, output.size(2))
            chunk_target = chunk_target.reshape(-1)
            loss = criterion(output, chunk_target)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

    epoch_loss /= num_batches
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Update the learning rate based on the epoch loss
    scheduler.step(epoch_loss)


train len:  8633346
valid len:  4525239
Epoch [10/5000], Loss: 5.5386
Epoch [20/5000], Loss: 5.4542
Epoch [30/5000], Loss: 5.4490
Epoch [40/5000], Loss: 5.4462
Epoch [50/5000], Loss: 5.4345
Epoch [60/5000], Loss: 5.4241
Epoch [70/5000], Loss: 5.4196
Epoch [80/5000], Loss: 5.4138
Epoch [90/5000], Loss: 5.4149
Epoch [100/5000], Loss: 5.4143
Epoch [110/5000], Loss: 5.4095
Epoch [120/5000], Loss: 5.4006
Epoch [130/5000], Loss: 5.4113
Epoch [140/5000], Loss: 5.3985
Epoch [150/5000], Loss: 5.3948
Epoch [160/5000], Loss: 5.3997
Epoch [170/5000], Loss: 5.4008
Epoch [180/5000], Loss: 5.3906
Epoch [190/5000], Loss: 5.3897
Epoch [200/5000], Loss: 5.3887
Epoch [210/5000], Loss: 5.4044
Epoch [220/5000], Loss: 5.3958
Epoch [230/5000], Loss: 5.3968
Epoch [240/5000], Loss: 5.3842
Epoch [250/5000], Loss: 5.3705
Epoch [260/5000], Loss: 5.3775
Epoch [270/5000], Loss: 5.3843
Epoch [280/5000], Loss: 5.3791
Epoch [290/5000], Loss: 5.3737
Epoch [300/5000], Loss: 5.3857
Epoch [310/5000], Loss: 5.3714
Epoch [3

In [15]:
#torch.save(awd_lstm.state_dict(), './AWD_LSTM_generator_V3.pth')

In [11]:
#to used the saved model
saved_model_path = "AWD_LSTM_generator_V3.pth"
awd_lstm = AWDLSTM(input_size, hidden_size, num_layers, embedding_size=embedding_size, dropout=dropout, weight_dropout=weight_dropout).to(device)
awd_lstm.load_state_dict(torch.load(saved_model_path))

<All keys matched successfully>

In [21]:
hidden = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
           torch.zeros(num_layers, batch_size, hidden_size).to(device))

In [ ]:
generated_text = generate(awd_lstm, prime_str='Th', predict_len=1000, vocab_mapping=char_to_idx, inv_vocab_mapping=idx_to_char, temperature=1.0)
print(generated_text)